In [1]:
import six
import sys
sys.modules['sklearn.externals.six'] = six

In [2]:
import mlrose
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from time import process_time

In [11]:
print("Running FourPeaks...")

#fitness = mlrose.FlipFlop()
#problem = mlrose.DiscreteOpt(100, fitness)
fitness = mlrose.FourPeaks(t_pct=0.1)
problem = mlrose.DiscreteOpt(length=100, fitness_fn=fitness, maximize=True, max_val=2)

RANDOM_SEED = 42
MAX_ATTEMPTS = 500

Running FourPeaks...


In [4]:
#%% tuning for SA
curve_list = []
decays = [0.999, 0.99, 0.9]
for d in decays:
    schedule = mlrose.GeomDecay(decay=d)
    _, _, curve = mlrose.simulated_annealing(
        problem,
        schedule=schedule,
        max_attempts=MAX_ATTEMPTS,
        max_iters=500,
        curve=True,
        random_state=RANDOM_SEED,
    )
    curve_list.append(curve)
    
df = pd.DataFrame(curve_list).transpose()
df.columns = decays
df.plot()
plt.xlabel("Iteration")
plt.ylabel("Fitness")
plt.title("FourPeaks: Fitness curve vs decay rate in SA")
plt.savefig("output/fourpeaks_sa_decay.png")
plt.close()

print(df.max())

Running FourPeaks...
0.999    13.0
0.990    13.0
0.900    13.0
dtype: float64


/Users/chux/Desktop/mlrose/mlrose/algorithms.py:311: RuntimeWarning: overflow encountered in exp
  prob = np.exp(delta_e/temp)


In [23]:
#%% tuning for HillClimbing
curve_list = []
max_attempts=[50,200,500]
#fitness = mlrose.FlipFlop()
#problem1 = mlrose.ContinuousOpt(100, fitness)

for r in max_attempts:
    _, _, curve = mlrose.random_hill_climb(
    problem,
    max_attempts=r,
    max_iters=500,
    curve=True,
    random_state=RANDOM_SEED,
    )
    print(curve)
    curve_list.append(curve)
    
df = pd.DataFrame(curve_list).transpose()
df.columns = restarts
df.plot()
plt.xlabel("Iteration")
plt.ylabel("Fitness")
plt.title("FourPeaks: Fitness curve vs max attempts in Hill Climbing")
plt.savefig("output/fourpeaks_rhc_maxattempts.png")
plt.close()

print(df.max())

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
 4. 

In [5]:
#%% tuning for GA
curve_list = []
pop_sizes = [50, 100, 200, 400]
for p in pop_sizes:
    _, _, curve = mlrose.genetic_alg(
        problem,
        max_attempts=MAX_ATTEMPTS,
        max_iters=500,
        pop_size=p,
        curve=True,
        random_state=RANDOM_SEED,
    )
    curve_list.append(curve)

df = pd.DataFrame(curve_list).transpose()
df.columns = pop_sizes
df.plot()
plt.xlabel("Iteration")
plt.ylabel("Fitness")
plt.title("FourPeaks: Fitness curve vs population size in GA")
plt.savefig("output/fourpeaks_ga_pop.png")
plt.close()

print(df.max())

50     113.0
100    115.0
200    122.0
400    123.0
dtype: float64


In [7]:
#%% tuning for MIMIC
curve_list = []
nth_pct = [0.2, 0.4]
for p in nth_pct:
    _, _, curve = mlrose.mimic(
        problem,
        max_attempts=MAX_ATTEMPTS,
        max_iters=50,
        keep_pct=p,
        curve=True,
        random_state=RANDOM_SEED,
    )
    curve_list.append(curve)

df = pd.DataFrame(curve_list).transpose()
df.columns = nth_pct
df.plot()
plt.xlabel("Iteration")
plt.ylabel("Fitness")
plt.title("FourPeaks: Fitness curve vs nth percentile in MIMIC")
plt.savefig("output/fourpeaks_mimic_nth.png")
plt.close()

print(df.max())



0.2    118.0
0.4    117.0
dtype: float64


In [6]:
#%% Putting together
curve_list = []
time_list = []
n_eval = []
algo_list = ["RHC", "SA", "GA", "MIMIC"]

# RHC
t1 = process_time()
_, _, curve = mlrose.random_hill_climb(
    problem,
    max_attempts=MAX_ATTEMPTS,
    max_iters=500,
    curve=True,
    random_state=RANDOM_SEED,
)
t2 = process_time()
time_list.append((t2 - t1) / len(curve))
curve_list.append(curve)
n_eval.append(np.argmax(curve) + 1)

# SA
t1 = process_time()
_, _, curve = mlrose.simulated_annealing(
    problem,
    max_attempts=MAX_ATTEMPTS,
    max_iters=500,
    curve=True,
    random_state=RANDOM_SEED,
)
t2 = process_time()
time_list.append((t2 - t1) / len(curve))
curve_list.append(curve)
n_eval.append(np.argmax(curve) + 1)

# GA
t1 = process_time()
_, _, curve = mlrose.genetic_alg(
    problem, max_attempts=MAX_ATTEMPTS, curve=True, random_state=RANDOM_SEED,
)
t2 = process_time()
time_list.append((t2 - t1) / len(curve))
curve_list.append(curve)
n_eval.append((np.argmax(curve) + 1) * 200)

# MIMIC
t1 = process_time()
_, _, curve = mlrose.mimic(
    problem,
    max_attempts=MAX_ATTEMPTS,
    keep_pct=0.4,
    curve=True,
    random_state=RANDOM_SEED,
)
t2 = process_time()
time_list.append((t2 - t1) / len(curve))
curve_list.append(curve)
n_eval.append((np.argmax(curve) + 1) * 200)

df = pd.DataFrame(curve_list).transpose()
df.columns = algo_list
df.plot()
plt.xlabel("Iteration")
plt.ylabel("Fitness")
plt.title("FourPeaks: Fitness curve vs algorithms")
plt.savefig("output/fourpeaks_algo.png")
plt.close()

print("time per iteration:")
print(time_list)
print("number of func eval reaching maxima:")
print(n_eval)
print("maxima reached:")
print(df.max())

time per iteration:
[2.781081081081417e-05, 4.398199999999974e-05, 0.020482734999999995, 4.7654325]
number of func eval reaching maxima:
[85, 242, 20000, 4800]
maxima reached:
RHC        4.0
SA        13.0
GA       122.0
MIMIC    117.0
dtype: float64
